In [137]:
import pandas as pd

from mapping.utils.processing import preprocess

In [138]:
matches_firms = pd.read_csv('../data/matches_firms.csv')
matches_funds = pd.read_csv('../data/matches_funds.csv')
matches_owners = pd.read_csv('../data/matches_owners.csv')

In [139]:
matches_firms_funds = pd.merge(matches_firms, matches_funds, how='inner')

In [140]:
# Formatting Data Frame
filter_columns = [
    'form_d_fund_id', 'cik_no_fund', 'fund',
    'form_adv_firm_id', 'crd_no_firm', 'matched_firm', 'firm_confidence',
    'form_adv_fund_id', 'crd_no_fund', 'matched_fund', 'fund_confidence',
]
matches_firms_funds = matches_firms_funds[filter_columns]

In [141]:
matches_firms_funds.head()

,form_d_fund_id,cik_no_fund,fund,form_adv_firm_id,crd_no_firm,matched_firm,firm_confidence,form_adv_fund_id,crd_no_fund,matched_fund,fund_confidence
0,54568,1694312,Ajo Emerging Markets Small Cap Fund,NaN,309921,Capitalworks Emerging Markets Advisor,0.985133,602.0,105545,Ajo Emerging Markets Small Cap Fund,0.000000
1,60070,1685357,Libra Fossil Fuel Free Fund,NaN,281914,Fund,0.927080,11352.0,294197,Libra Fossil Fuel Free Fund,0.000000
2,38363,1679614,Fiam Global Low Volatility Equity Fund,NaN,137649,Global Equity Advisors,0.891444,10035.0,133196,Fiam Global Low Volatility Equity Fund,0.000000
3,43377,1753457,Parian Global Us Fund,20874.0,297279,Parian Global Management,0.819455,20874.0,297279,Parian Global Us Fund Ii,0.263488
4,24565,1450552,Davi Luxury Brand Group,142265.0,306533,Daventry Group,0.983191,3170.0,110885,Capital Group Euro Bond Fund Lux,0.974050


Testing

In [142]:
match_df = matches_firms_funds

In [143]:
cols = ['cik_no_fund', 'crd_no_firm', 'crd_no_fund']

match_df = match_df[cols]

In [144]:
from fuzzywuzzy import fuzz

def get_match_df(data_a, data_b):

    left_values = list(set(data_a.related_partners))
    right_values = data_b.direct_owners.to_list()

    matches = []

    for partner in left_values:
        best_owner, best_raio = None, 0
        for owner in right_values:
            ratio = fuzz.SequenceMatcher(None, partner, owner).ratio()
            if ratio > best_raio:
                best_owner = owner
                best_raio = ratio
        matches.append((partner, best_owner, best_raio))

    return pd.DataFrame(matches, columns=['related_partner', 'direct_owners_owner', 'owners_ratio'])

In [145]:
related_partners = pd.read_pickle('../storage/related_partners.pkl')
direct_owners = pd.read_pickle('../storage/direct_owners.pkl')

related_partners = preprocess(related_partners, related_partners=True)
direct_owners = preprocess(direct_owners, direct_owners=True)

result_df = []

for idx, row in match_df.iterrows():

    cik_no_fund, crd_no_firm, crd_no_fund = row.cik_no_fund, row.crd_no_firm, row.crd_no_fund

    table_a = related_partners[related_partners.cik_no_related_partners == cik_no_fund]
    table_b = direct_owners[(direct_owners.crd_no_owners == crd_no_firm) | (direct_owners.crd_no_owners == crd_no_fund)]

    match = get_match_df(table_a, table_b)

    match['cik_no_fund'] = cik_no_fund
    match['crd_no_firm'] = crd_no_firm
    match['crd_no_fund'] = crd_no_fund

    result_df.append(match)

In [146]:
stack_result = pd.DataFrame()

for data in result_df:
    if not data.empty:
        if not stack_result.empty:
            stack_result = pd.concat([stack_result, data], join='inner', ignore_index=True)
        else:
            stack_result = data

In [148]:
stack_result.describe()

,owners_ratio,cik_no_fund,crd_no_firm,crd_no_fund
count,4086.000000,4.086000e+03,4086.000000,4086.000000
mean,0.376562,1.200393e+06,191189.817425,190179.048458
std,0.103952,4.029254e+05,84400.335448,73317.742681
min,0.000000,3.570000e+03,361.000000,361.000000
25%,0.312500,1.075202e+06,111289.000000,137432.000000
50%,0.363636,1.355848e+06,160946.500000,163297.000000
75%,0.424242,1.460329e+06,290576.000000,283533.000000
max,1.000000,1.769153e+06,324330.000000,324515.000000
